In [7]:
from pynq import Overlay
from pynq import MMIO
Max_buffer_size = 345600
Clefia = Overlay("./bitstream/256/design_enc.bit")

phys_addr = Clefia.ip_dict["clefia_enc_0"]["phys_addr"]
addr_range = Clefia.ip_dict["clefia_enc_0"]["addr_range"]
mmio = MMIO(phys_addr, addr_range)

#Clefia.ip_dict #offsetの確認

In [8]:
def call_Clefia(pt, skey):
    mmio.write(16,bytes(pt))            # Memory_pt: 16
    mmio.write(32,bytes(skey))          # Memory_skey: 32
    mmio.write(0,1)                     # ap_start: 0
    while not (mmio.read(0) & 0x02):
        pass
        
    return 0



In [9]:
import time
# byte型はmmioに一度ですべて書き込める
pt = bytes([0x00,0x01,0x02,0x03,0x04,0x05,
            0x06,0x07,0x08,0x09,0x0a,0x0b,
            0x0c,0x0d,0x0e,0x0f])

skey = bytes([0xff,0xee,0xdd,0xcc,0xbb,0xaa,0x99,0x88,
         0x77,0x66,0x55,0x44,0x33,0x22,0x11,0x00,
         0xf0,0xe0,0xd0,0xc0,0xb0,0xa0,0x90,0x80,
         0x70,0x60,0x50,0x40,0x30,0x20,0x10,0x00])
         
print(pt)  # x09, x0a, x0dはASCIIで置き換えられる（数値に影響はなし）

NUM_OF_LOOPS = 1<<17
i = 0

# -----計算開始-----
time_start = time.perf_counter_ns()
for i in range(0,NUM_OF_LOOPS,1):
    call_Clefia(pt, skey)

time_end = time.perf_counter_ns()
# -----計算終了-----

time_took = time_end - time_start
time_ave = time_took / NUM_OF_LOOPS


print(f"Average:{time_ave}ns")



b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f'
Average:155807.65116882324ns


In [ ]:
# ------ ここから下はメモとデモ -----

In [35]:
def call_Clefia(pt):
    temp = 0
    enc = []
    print("\n-----処理時間計測開始-----") #消す
    time_start = time.perf_counter_ns()
    mmio.write(16,bytes(pt))            # Memory_pt: 16
    print("計算前：", mmio.read(CTRL)) #消す
    mmio.write(0,1)                     # ap_start: 0
    while not mmio.read(0x00 & 0x04) == 0x04: 
        print("計算中：", mmio.read(CTRL)) #消す
        print("waiting...") # passに変える
        
    print("計算後：", mmio.read(CTRL)) #消す
    time_end = time.perf_counter_ns()
    print("\n-----処理時間計測終了-----") #消す
    print("計算時間：", time_end-time_start, "s.")
    for i in range(0,4): #一度に１６バイトの読み込みはできない
        enc.append(mmio.read(32+(i*4)).to_bytes(4,"little"))  #Memory_ct: 32
          
    print("ciphertext:",enc)
    return 0



In [ ]:
string = "abcdefgh" #Max 16 Bytes & Multiple of 4
print(len(string.encode()))
string = bytes(string.encode())
key_bitlen = 128

call_Clefia(string, key_bitlen)

"""stringへの入力が4バイト単位しか受け付けない
   空白文字を入れるなどの処理をする？？？"""
"""ポートのサイズを拡大したらもっと処理できる？？
Pythonで16バイトずつ区切って何度か呼び出す？"""

"""VITISでシミュレーションした時と暗号化の中身が異なる。
   復号は正しく行えている。
   mmio classで書き込む際にnumpy形式に書き換えられているのが原因の可能性大
   エンディアン
"""

"""128 -> 128 OK.    128 -> 192 OK.    128 -> 256 OK.
   192 -> 128 NG.    192 -> 192 OK.    192 -> 256 OK.
   256 -> 128 NG.    256 -> 192 OK.    256 -> 256 OK.
   初回起動時もバグる？"""